# Back-Substitution and Gaussian Elimination

## Back-Substitution:
Back substitution is an algorithm that can be used to solve linear systems with upper-triangular matrices ($Ux = b$, where U is an upper-triangular matrix). This is the same algorithm you learned in your linear algebra class to solve linear systems by hand, only now we are putting it into code which can be used to solve much larger systems.

I won't explain the whole process here since you should have covered it in previous classes, but in case you didn't, or you just forgot, this is a quick video explaining the process. The back substitution starts at about  4:14, but notice that in the video he has already made all the leading coefficients equal to 1 (which isn't necessary for back substitution to work).

https://www.youtube.com/watch?v=jOC4fMgl7TI


The main idea is that after you have row-reduced your matrix, you have an upper-diagonal matrix, which we call $U$. From there, the last element in the vector $x$ is easily solved for by dividing the last element of $b$ by the only non-zero element in the last row of $U$. Then you use that value to solve for the next element up in $x$ and continue solving upwards until you have all the values.

We can first formalize this algorithm into the following equation (from the lecture slides) and then implement it as code:

$$x_{i} = (b_{i} - \sum_{j=i+1}^{n} u_{ij}x_{j})/u_{ii}$$


#### Note that a similar algorithm, called forward substitution, exists for solving systems with lower-triangular matrices.


In [1]:
import numpy as np

### Question 1: 

Finish implementing the back substitution algorithm below

In [1]:
def backward_substitution(U, b):
    """Return a vector x with np.matmul(U, x) == b, where 
        * U is an nxn numpy matrix that is upper-triangular and non-singular
        * b is an nx1 numpy vector
    """
    n = U.shape[0]  # get the height of matrix U (square matrix)
    x = np.zeros_like(b, dtype=np.float)  # x has the same shape as b, initialize it with zeros
    
    for i in range(_____, _____, _____):  # looping through rows backwards
        
        # for loop for the summation:
        s = 0
        for j in range(_____, _____, _____):  # loop through columns backwards, but stop once the diagonal is reached
            s += _____
            
        x[i] = _____
        
        
    return x

In [ ]:
# create some linear system for testing (make sure A satisfies the conditions in the docstring of the function above)
A = _____
b = _____


# running the solution
x = backward_substitution(A, b)

print("A:\n", A)
print("b: ", b)
print("\nx: ", x)

print("Ax: ", np.matmul(A, x))  # if your code works, this should give the same thing as b

## Gaussian Elimination:

Let's go through (part of) an example of Gaussian Elimination to help give some idea of how to code the algorithm.

Consider the system $Ax = b$ where 

$A = \begin{bmatrix}
1 & 2 & 3\\
4 & 5 & 6\\
7 & 8 & 8
\end{bmatrix}    $

and

$b = \begin{bmatrix}
1 \\
1 \\
1 
\end{bmatrix}$

The reason I didn't make the last element in $A$ a 9 is that it would make the matrix singular (determinant=0).

The first step in the row-reduction process is to make all elements in column 0 equal to zero if they are below the diagonal. Lets start with the element in the row 1: $A_{1,0}$. We can do this by multiplying a row by some factor, and then subtracting it from row 1. We have several rows to choose from, but let's use the row that has the same index as the column we are trying to eliminate (ie. we are eliminating column 0, so let's use row 0 for the row-reduction). This may seem arbitrary, but it keeps things simple if we always choose the row this way, and it ensures that later in the process we don't accidentally change any of the columns that have already been eliminated.

So our first row reduction will be:

#### row_1 = row_1 - m*row_0

By looking at the matrix, we can see that if we choose m=4, we will get the desired result of $A_{1,0} = 0$. In general, m is the ratio between the element you are trying to eliminate, and the element in the same column of the row you are using to eliminate it (in this case m = 4/1).

When doing this by hand, you typically use an augmented matrix, where $b$ is treated sort of like a column of $A$. Therefore, we need to do the same operation on b:

#### b_1 = b_1 - 4*b_0


Now we can eliminate the next row:

#### row_2 = row_2 - 7*row_0 

#### b_2 = b_2 - 7*b_0

Notice our m value is now 7/1, and we are still using row 0 for the row-reduction since we are working on eliminating column zero.

Now that column 0 is eliminated, we just repeat the process for the remaining columns (excluding the last  column, because there is nothing below the diagonal there).

### Question 2:

Finish implementing the function "eliminate(A, b, k)" below. This function implements part of the row-reduction process. In particular, it eliminates row k. That is, it makes all elements in row k equal to zero if they are below the diagonal, using the row-reduction techniques you have learned in your linear algebra class, and modifies $b$ accordingly.

This function is equivalent to creating the elimination matrix $M_k$ and multiplying it to both $A$ and $b$.

In [24]:
def eliminate(A, b, k):
    """Eliminate the k-th column of A, in the system np.matmul(A, x) == b,
    so that A[i, k] = 0 for i > k. The elimination is done in place.
    
    In other words, perform valid row-reduction operations such that all the elements in column k that are
    below the diagonal end up as zero.
    """
    n = A.shape[0]
    
    # we want to eliminate all the values in column k that are below the diagonal
    
    for i in range(_____, _____):  # loop through rows below the diagonal
        m = _____  # find the factor needed to eliminate the element A[i,k], using the kth row
        
        ________  # perform the row reduction
        
        ________  # apply the same change to the corresponding element in b
        

Now we can test our code. The function below loops through all columns, eliminating them one at a time using the function implemented above, and then uses back substitution to solve the system.

In [25]:
def gauss_elimination(A, b):
    """Return a vector x with np.matmul(A, x) == b using
    the Gauss Elimination algorithm, without partial pivoting."""
    
    for k in range(A.shape[0] - 1):  # eliminate all columns (make them zero below the diagonal)
        eliminate(A, b, k)
    x = backward_substitution(A, b)  # now that you have a diagonal matrix, use back-sub to solve the system
    return x

In [1]:
# Create a linear system to test

# note that eliminate(A, b, x) modifies A and b, so make copies of them that you can youse to check if Ax=b

## A note on partial pivoting:

The algorithm we implemented does not include partial pivoting. Partial pivoting involves swapping rows during the elimnation process, so that the largest number in the column is used for the division in the equation for m. In other words, it ensures that the denominator in the calculation of m = a/b is the largest available number in the column, to avoid dividing by very small numbers. This prevents m from blowing up, as it will always be less than 1. In turn, this prevents the loss of precision that may arise from adding a small number to a very large number.